## Apply best model on test (submit) data

## Import

In [7]:
import pytesseract
import cv2
import os
import ntpath
import pandas as pd
import json
from datasets import load_dataset

from functions.add_to_file import add_to_file 

## Read data

In [25]:
data_files = {"train": "train_dataset.csv", "val": "val_dataset.csv", "test": "test_dataset.csv"}
dataset = load_dataset("../../data/input", data_files=data_files)

Using custom data configuration input-7726882c3a6b36be
Reusing dataset csv (/home/mayara/.cache/huggingface/datasets/csv/input-7726882c3a6b36be/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)
100%|██████████| 3/3 [00:00<00:00, 674.47it/s]


In [26]:
labels = {0: 'B-address', 1: 'B-company', 2: 'B-date', 3: 'B-total', 4: 'O'}

label_colors = {'address': 'blue', 'company': 'green',
               'date': 'red', 'total': 'violet', 'other': 'gray'}

1) OCR textos ruins
2)

In [30]:
dataset['train'].to_pandas()

,id,company,date,address,total,text
0,X51005717526,GERBANG ALAF RESTAURANTS SDN BHD,02/03/2018,"LEVEL 6, BANGUNAN TH, DAMANSARA UPTOWN3 NO.3, ...",28.50,[/ 4 Aljaf Restaurants Sdn Bhd (OES 1-M) forme...
1,X51006334926,GOLDEN ARCHES RESTAURANTS SDN BHD,25/12/2016,"LEVEL 6, BANGUNAN TH, DAMANSARA UPTOWN3 NO.3, ...",29.30,Your order number 1s 264 Golden Arches Restaur...
2,X51008164969,ONE ONE THREE SEAFOOD RESTAURANT SDN BHD,14-06-2018,"NO. 1, TAMAN SRI DENGKIL, JALAN AIR HITAM 4380...",59.00,. ONE THREE SEAFOOD RESTAURANT SON BHD (112090...
3,X51006556646,GARDENIA BAKERIES (KL) SDN BHD,21/08/2017,"LOT 3, JALAN PELABUR 23/1, 40300 SHAH ALAM, SE...",73.55,meee nature “Yowiet regu : “ACCOUNT'S COPY |...
4,X51006882603,ECO-SHOP MARKETING SDN BHD,26/02/2018,"LOT F1-01, 6, JALAN 8/27A, SEKSYEN 5 WANGSA MA...",16.30,TAX INVOICE Eco-Shop Marketing Sdn Bhd GST Reg...
...,...,...,...,...,...,...
607,X51005749905,ENW HARDWARE CENTRE (M) SDN BHD,10/03/2018,"NO: G3, BLK G, JIN PJU 1A/3, ARA DAMANSARA, 47...",34.00,\ ‘ ENW Hardware Centre (M) Sdn. Bhd. Co. Reg....
608,X51007225417,ENW HARDWARE CENTRE (M) SDN. BHD.,09/04/2018,"NO: G3, BLK G, JLN PJU 1A/3, ARA DAMANSARA, 47...",51.00,\ baht é ENW Hardwa. 2 Centre (M) Sdn. Bhd. Co...
609,X51005757349,GOLDEN KEY MAKER,24-MAR-2018,"NO 5, JALAN KENARI 2, BANDAR PUCHONG JAYA, 471...",21.00,31802053 = 4 . GOLDEN KEY MAKER (000760274-K) ...
610,X51007339643,AIK HUAT HARDWARE ENTERPRISE (SETIA ALAM) SDN BHD,30/03/2017,"NO. 17-G, JALAN SETIA INDAH (X) U13/X, SETIA A...",6.00,set LT HULL & IN CRIGIEAL 03 o : q ...


In [ ]:
df = pd.DataFrame()
df['x'] = dataset['train']['text']
df['y'] = 'COMPANY'
df

In [ ]:
df = pd.DataFrame()
df['x'] = dataset['train']['company']
df['y'] = 'COMPANY'
df

In [20]:
df = pd.DataFrame()
df['x'] = dataset['train']['company']
df['y'] = 'COMPANY'
df

,x,y
0,GERBANG ALAF RESTAURANTS SDN BHD,COMPANY
1,GOLDEN ARCHES RESTAURANTS SDN BHD,COMPANY
2,ONE ONE THREE SEAFOOD RESTAURANT SDN BHD,COMPANY
3,GARDENIA BAKERIES (KL) SDN BHD,COMPANY
4,ECO-SHOP MARKETING SDN BHD,COMPANY
...,...,...
607,ENW HARDWARE CENTRE (M) SDN BHD,COMPANY
608,ENW HARDWARE CENTRE (M) SDN. BHD.,COMPANY
609,GOLDEN KEY MAKER,COMPANY
610,AIK HUAT HARDWARE ENTERPRISE (SETIA ALAM) SDN BHD,COMPANY


## Preprocessing

Transform to IOB format

In [22]:
# Loading the libraries
from nltk.corpus.reader import ConllChunkCorpusReader
 
# Initializing
reader = ConllChunkCorpusReader('.', r'.*\.iob', ('COMPANY'))
 
reader.chunked_words()
 
reader.iob_words()

ValueError: concat() expects at least one object!

Transform None type into string

In [20]:
train_corpus.fillna("",inplace=True)
test_corpus.fillna("",inplace=True)

Transform text column to string upper case

In [21]:
train_corpus['text'] = train_corpus['text'].astype(str).str.upper()
test_corpus['text'] = test_corpus['text'].astype(str).str.upper()

In [23]:
test_corpus['text'].head()

0    NOEONF UNIHAKKA INTERNATIONAL SDN BHD 07 JUN 2...
1      PASARAYA BORONG PINTAR SDN BHD BR NO.: (1245...
2    / SUPER SEVEN CASH & TH. SDN BHD WALE 590 150-...
3    ’ TORE : \O SEMBOYAN TEGAS SDN BHD WE” NO.5 & ...
4    A03 (26 CONTENTO (JNO761170-4) 15, JALAN PERHA...
Name: text, dtype: object

## Add data to .txt file

In [61]:
test_corpus.head()

,id,text,company,date,address,total
0,X51007846290,NOEONF UNIHAKKA INTERNATIONAL SDN BHD 07 JUN 2...,UNIHAKKA INTERNATIONAL SDN BHD,20 JUN 2018,"12, JALAN TAMPOI 7/4,KAWASAN PERINDUSTRIAN TAM...",RM8.20
1,X51005447859,PASARAYA BORONG PINTAR SDN BHD BR NO.: (1245...,PASARAYA BORONG PINTAR SDN BHD,20180304,"NO 19-G& 19-1& 19-2 JALAN TASIK UTAMA 4, MEDAN...",3.20
2,X51005746203,/ SUPER SEVEN CASH & TH. SDN BHD WALE 590 150-...,SUPER SEVEN CASH & CARRY SDN BHD,24-03-18,NO. 1 JALAN EURO 1 OFF JALAN BATU TIGA SUNGAI ...,RM367.10
3,X51006329183,’ TORE : \O SEMBOYAN TEGAS SDN BHD WE” NO.5 & ...,GOLDEN KEY MAKER,24-03-18,"NO. 53, JALAN BESAR, 45600 BATANG BERJUNTAI SE...",6.00
4,X51005442388,"A03 (26 CONTENTO (JNO761170-4) 15, JALAN PERHA...",MR. D.I.Y. (M) SDN BHD,24-03-18,"NO. 17-G, JALAN SETIA INDAH (X) U13/X, SETIA A...",28.00


In [68]:
submit_path = '../data/output/model_nlp_dataset_submit'

In [69]:
# Create the directory for results
os.mkdir(path=submit_path)

In [70]:
for id, row in test_corpus.iterrows():
    data = {}
    data['company'] = row.company
    data['address'] = row.address
    data['total'] = row.total
    data['date'] = row.date
    
    add_to_file(data, submit_path, row.id+'.jpg')

Calculated!{"recall": 0.335014409221902, "precision": 0.335014409221902, "hmean": 0.335014409221902}%